In [1]:
import cvxpy
from matplotlib import pyplot as plt
import numpy as np
from src.components.microgrid_simple import SimpleMicrogrid
from src.components.battery import Battery, BatteryParameters
from src.rl.a2c.c_mg_simple import set_all_seeds

/home/roberto.guillen/.conda/envs/bcte/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Set env vars

In [2]:
set_all_seeds(0)
n = 24

# Same env obs as test simple env
mg = SimpleMicrogrid(steps=n, min_temp=29, max_temp=31, peak_pv_gen=1, peak_grid_gen=1, peak_load=1, disable_noise=True)
demand = mg.demand
pv = mg.pv_gen
price_s = mg.price
emission = mg.emission
price_b = price_s / 4

In [3]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
    
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

0.8200000000000001

### Adv battery calc (not working)

In [10]:
set_all_seeds(0)
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't
# for i in range(n):
#     constraints.append(action[i] <= 0.9)
#     constraints.append(action[i] >= -0.9)

# Transition
obj = 0

for i in range(n):
    # Max and min battery charge 
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery SOC
    # self.soc = self.soc + (p_charge * self.efficiency - p_discharge / self.efficiency) / self.capacity

    constraints.append(battery[i+1] == battery[i] + ((action[i] * battery_params["efficiency"])/battery_params["efficiency"])/battery_params["capacity"] )
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value, consumption.value, price_s

(1.031089160990395,
 array([0.1       , 0.35510302, 0.45092906, 0.50774134, 0.56838129,
        0.67959444, 0.9       , 0.8086571 , 0.1       , 0.28634816,
        0.89236155, 0.58138849, 0.50825121, 0.44613645, 0.21550249,
        0.80113181, 0.9       , 0.69301569, 0.20327912, 0.40546626,
        0.42948302, 0.3730126 , 0.32421838, 0.24404732, 0.1       ]),
 array([ 0.25510302,  0.09582604,  0.05681228,  0.06063995,  0.11121315,
         0.22040556, -0.0913429 , -0.7086571 ,  0.18634816,  0.60601339,
        -0.31097305, -0.07313728, -0.06211477, -0.23063396,  0.58562932,
         0.09886819, -0.20698431, -0.48973657,  0.20218715,  0.02401675,
        -0.05647042, -0.04879422, -0.08017106, -0.14404732]),
 array([ 5.10568989e-01,  3.70547220e-01,  3.33186321e-01,  3.35473622e-01,
         3.78564927e-01,  6.81282887e-01,  6.45423142e-01,  1.15869241e-12,
        -1.13460414e-11, -3.27555693e-11, -9.32041223e-01, -7.79844435e-01,
        -7.63900645e-01, -8.41662837e-01, -4.64221443e-1

### Real dataset

In [11]:
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery
    # Starts in 0.1
constraints.append(battery[0] == battery_params["soc_min"])
    # Max and min batteries
for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition
obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    # Update battery
    constraints.append(battery[i+1] == battery[i] + action[i] * battery_params["efficiency"])
    # Update net 
    constraints.append(consumption[i] == demand[i]-pv[i] + action[i] * battery_params["efficiency"])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

(1.5289098310943403,
 array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
 array([ 4.34418145e-12,  4.34754569e-12,  3.15528729e-12,  2.16153858e-12,
         1.95456966e-12,  2.39877078e-12,  3.02787391e-12,  3.94100745e-12,
         3.60826551e-12,  2.87791336e-12,  2.76885584e-12,  2.41457816e-12,
         2.05170996e-12,  1.68823588e-12,  1.24054002e-12,  1.47353772e-12,
        -4.36544664e-12, -2.50470417e-12, -1.50923010e-12, -2.27653773e-12,
        -3.36145908e-12, -5.44275885e-12, -8.90133694e-12, -1.58494219e-11]))

### Test Dataset

In [7]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4

In [8]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

(0.7090152143676689,
 array([9.62456752e-15, 2.15284679e-11, 3.00000000e-01, 7.52271014e-12,
        2.45946399e-13]),
 array([ 2.15199519e-11,  3.00000000e-01, -3.00000000e-01, -7.27672641e-12]))

In [9]:


x_hat = cvxpy.Variable(A.shape[1])
objective = cvxpy.Minimize(cvxpy.norm(x_hat, 1))
    constraints = [A * x_hat == y]
    prob = cvxpy.Problem(objective, constraints)
    prob.solve()
    return np.squeeze(np.array(x_hat.value)), (prob.value, prob.status)

IndentationError: unexpected indent (812586873.py, line 3)